# Fifth Agent Agent

Introducing a critical agent - the agent that brings it all together.

# Planning Agent

There are a number of frameworks out there that support building Agentic Workflows.

OpenAI has OpenAI Agents SDK, LangChain has LangGraph, and there's Autogen from Microsoft, Crew.ai and many others.  

Each of these are abstractions on top of APIs to LLMs; some are lightweight, others come with significant functionality.

It's also perfectly possible - and sometimes considerably easier - to build an agentic solution by calling LLMs directly.

There's been considerable convergence on LLM APIs, and it's not clear that there's a need to sign up for one of the agent ecosystems for many use cases.

Anthropic has an [insightful post](https://www.anthropic.com/research/building-effective-agents) on building effective Agentic architectures that's well worth a read.

# We are going to use OpenAI Agents SDK for this Agent

## And we're using Tools to give our Agent autonomy

In our case, we're going to create an Agent that uses Tools to make decisions about what to do next.

Let's see how it works

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
!pip install openai-agents

In [2]:
from agents import Agent, Runner, trace, function_tool

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


In [3]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o"


OpenAI API Key exists and begins sk-proj-


In [4]:
# Use the Scanner agent from before

from price_agents.scanner_agent import ScannerAgent
scanner = ScannerAgent()

# Our tools

The next 3 cells have 3 **fake** functions that we will allow our LLM to call

In [5]:
@function_tool
def scan_the_internet_for_bargains():
    """ This tool scans the internet for great deals and gets a curated list of promising deals """
    print(f"Scanning the internet")
    results = scanner.test_scan()
    return results.model_dump()

In [6]:
@function_tool
def estimate_true_value(description: str) -> str:
    """ This tool estimates how much a product is actually work, given a text description of the product """
    print(f"Estimating true value of {description[:20]}...")
    return {"description": description, "estimated_true_value": 300}

In [7]:
@function_tool
def notify_user_of_deal(description: str, deal_price: float, estimated_true_value: float):
    """ This tool notified the user of a deal, given a description, a price and an estimated actual value """
    print(f"Notifying user of {description} which costs {deal_price} and estimate is {estimated_true_value}")
    return {"notification_sent": "ok"}

## Telling the LLM about the tools it can use, with JSON

"Tool calling" is giving an LLM the power to run code on your computer!

Sounds a bit spooky?

The way it works is a little more mundane. We give the LLM a description of each Tool and the parameters, and we let it inform us if it wants any tool to be run.

It's not like OpenAI reaches in and runs a function. In the end, we have an if statement that calls our function if the model requests it.

## OpenAI Agents SDK has made this easy for us

The decorator `function_tools` around each of our functions automatically generates the description we need for the LLM

In [8]:
tools = [scan_the_internet_for_bargains, estimate_true_value, notify_user_of_deal]
tools

[FunctionTool(name='scan_the_internet_for_bargains', description='This tool scans the internet for great deals and gets a curated list of promising deals', params_json_schema={'properties': {}, 'title': 'scan_the_internet_for_bargains_args', 'type': 'object', 'additionalProperties': False, 'required': []}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x118c23420>, strict_json_schema=True),
 FunctionTool(name='estimate_true_value', description='This tool estimates how much a product is actually work, given a text description of the product', params_json_schema={'properties': {'description': {'title': 'Description', 'type': 'string'}}, 'required': ['description'], 'title': 'estimate_true_value_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1190feac0>, strict_json_schema=True),
 FunctionTool(name='notify_user_of_deal', descripti

## And.. MCP

The Model Context Protocol from Anthropic is causing a lot of excitement.

It gives us a really easy way to integrate new capabilities with our agent, as more tools.

Here we will give our agent powers to write to our local filesystem in a directory "sandbox"

In [9]:
import os
from agents.mcp import MCPServerStdio
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))

# parameters describe an MCP server
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}


async with MCPServerStdio(params=files_params) as server:
    file_tools = await server.list_tools()

In [10]:
file_tools

[Tool(name='read_file', description='Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}),
 Tool(name='read_multiple_files', description="Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.", inputSchema={'type': 'object', 'properties': {'paths': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['paths'], 'additionalP

In [14]:
system_message = "You are an Autonomous AI Agent that makes use of tools to carry out your mission. Your mission is to find great deals on bargain products, and notify the user when you find them with a push notification and a written file."
user_message = "Your mission is to discover great deals on products. First you should scan the internet for bargain deals. Then for each deal, you should estimate its true value - how much it's actually worth. "
user_message += "Finally, you should pick the single most compelling deal where the deal price is much lower than the estimated true value, and "
user_message += "use your tools to send the user a push notification about that deal, and also use your tools to write or update a file called sandbox/deals.md with a description in markdown. "
user_message += "You must only notify the user about one deal, and be sure to pick the most compelling deal, where the deal price is much lower than the estimated true value. Then just respond OK to indicate success."
messages = [{"role": "system", "content": system_message},{"role": "user", "content": user_message}]

### And here's where it comes together - just 2 lines of code

In [15]:
async with MCPServerStdio(params=files_params) as server:
    agent = Agent(name="Planner", instructions=system_message, model="gpt-4o", tools=tools, mcp_servers=[server])
    result = await Runner.run(agent, user_message)

print(result)

Scanning the internet
Estimating true value of The Hisense R6 Serie...
Estimating true value of The Poly Studio P21 ...
Estimating true value of The Lenovo IdeaPad S...
Estimating true value of The Dell G15 gaming ...
Notifying user of The Poly Studio P21 is a 21.5-inch LED personal meeting display designed specifically for remote work and video conferencing. With a native resolution of 1080p, it provides crystal-clear video quality, featuring a privacy shutter and stereo speakers. This display includes a 1080p webcam with manual pan, tilt, and zoom control, along with an ambient light sensor to adjust the vanity lighting as needed. It also supports 5W wireless charging for mobile devices, making it an all-in-one solution for home offices. which costs 30.0 and estimate is 300.0
RunResult:
- Last agent: Agent(name="Planner", ...)
- Final output (str):
    OK
- 15 new item(s)
- 4 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more detail

## And now - putting all of this into a Planning Agent

In [16]:
from price_agents.autonomous_planning_agent import AutonomousPlanningAgent

In [17]:
import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [18]:
import chromadb
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [19]:
agent = AutonomousPlanningAgent(collection)

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is set up with DeepSeek
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready
INFO:root:[Specialist Agent] Specialist Agent is initializing - connecting to modal
INFO:root:[Specialist Agent] Specialist Agent is ready
INFO:root:[Messaging Agent] Messaging Agent is initializing
INFO:root:[Messaging Agent] Messaging Agent has initialized Pushover and Claude
INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is ready


In [20]:
result = agent.plan()

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is kicking off a run
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is calling scanner
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 15 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Output
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completi

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $699.99
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $500.00
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $399.99
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $350.00
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


...

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $899.99
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $700.00
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $399.00
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $349.00
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $199.99
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $299.00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is notifying user
INFO:root:[Messaging Agent] Messaging Agent is using Claude to craft the message
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:[Messaging Agent] Messaging Agent is 

### Check out the trace

https://platform.openai.com/traces

# Finally - with a Gradio UI

In [1]:
# Reset memory back to 2 deals discovered in the past

from deal_agent_framework import DealAgentFramework
DealAgentFramework.reset_memory()

In [ ]:
!python price_is_right.py

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Agent Framework] Initializing Agent Framework
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Agent Framework] Initializing Agent Framework
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Autonomous Planning Agent] Autonomous Planning Agent is initializing
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Autonomous Planning Agent] Autonomous Planning Agent is initializing
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Scanner Agent] Scanner Agent is initializing
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Scanner Agent] Scanner Agent is initializing
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Scanner Agent] Scanner Agent is ready
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Scanner Agent] Scanner Agent is ready
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Frontier Agent] Initializing Frontier Agent
[2025-04-09 19:49:14 -0400] [Agents] [INFO] [Front